# CISC451/893 Competition 1

**TEAM:**<br>
Ryan Elliott - 05999293<br>
Sizhe Guan 20090459<br>
Vishal Chotara 20307118
## Question 1<br>
### Accessibility of Kingston transit
City of Kingston has an extensive transit network. The city maintains 23 routes helping connect the east and west urban areas of the city. The urban regions of the city are well serviced by the transit network. The data environment is prepared in this cell. City of Kingston data is fetched using the API. The data evaluated is all the table of all transit stops, table of all driveways (commericial and residential), and table of active busroutes in Kingston. The effectiveness of a transit network is highly correlated to the systems accessibility. Accessibility can mean different things depending on the study. This analysis will define accessbility to be a walking distance of 200m. We will assume that if a bus stop is within 200m of a driveway/address then the stop will be accessible.

In [1]:
# The data environment is prepared in this cell. City of Kingston data is fetched
# using the API

import requests
import pandas as pd
import numpy as np
import json
stop_api_url = "https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=transit-gtfs-stops&q=&rows=799"
driveway_api_url = "https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=driveways&q=&rows=-1&facet=material"
busroute_api_url = 'https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=transit-gtfs-routes&q=&rows=-1&facet=route_short_name&exclude.route_long_name=999+Maintenance+-+Out+of+Service'
stop_response = requests.get(stop_api_url)
driveway_response = requests.get(driveway_api_url)
busroute_response = requests.get(busroute_api_url)

In [3]:
# Data must be unpacked using json() method.
x = stop_response.json()
x_driveway = driveway_response.json()
x_busroute = busroute_response.json()

# The json() frames are then turned into Pandas dataframes.
transit_stops = pd.json_normalize(x,'records')
driveways = pd.json_normalize(x_driveway,'records')
busroute = pd.json_normalize(x_busroute,'records')

The city actively maintains and updates the opendata the databases. Sometimes the new records do not have GPS coordinates. The records without GPS coordinates are removed from analysis.

In [5]:
# drops data if there is no GPS coordinates.
driveways = driveways.dropna(subset = ['fields.geo_point_2d'],how = 'all')

GPS coordinates are points on a sphere. Measuring the distance between poinst cannot be completed in cartesian space. The points must be projected onto a sphere to measure the arc length between the points. The haversin function is a common approximation used to measure the poits between two GPS coordinates.

In [6]:
import math

def haversine(coord1,coord2):
    # Takes two coordinate vectors coord1 and coord2 and 
    # returns the distance between two points in metres
    
    # Radius of the Earth at Kingston. 
    # Assuming mean elevation of 93m above sea level.
    R = 6367869
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a),math.sqrt(1-a))

In [8]:
transit_stops['bus_routes'] = pd.Series()

c:\users\ryan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


Next we evaluate the distance of the driveways to all the bus stops. An array is created of the distance of each driveway to each bus stop.

In [10]:
transit_lat = transit_stops['fields.stop_coordinates'].explode().to_numpy()[::2]
transit_long = transit_stops['fields.stop_coordinates'].explode().to_numpy()[1::2]
transit_coords = np.column_stack((transit_lat,transit_long))

driveway_lat = driveways['fields.geo_point_2d'].explode().to_numpy()[::2]
driveway_long =  driveways['fields.geo_point_2d'].explode().to_numpy()[1::2]

driveway_coords = np.column_stack((driveway_lat,driveway_long))

In [11]:
stop_distance = np.ndarray(shape = [driveway_lat.size,transit_lat.size])
k = 0
l = 0

for i in driveway_coords:
    l = 0
    for j in transit_coords:
        stop_distance[k,l] = haversine(i,j)
        l = l+1
    k = k+1

In [54]:
closest_stop = stop_distance.min(axis = 1)
print('median distance to a stop: ',round(np.median(closest_stop)),'m')
print('average distance to a stop: ',round(closest_stop.mean()),'m')

median distance to a stop:  199 m
average distance to a stop:  1287 m


Half of all the driveways in Kingston are within 200m of a bus stop. The average distance to a bus stop is heavily skewed by driveways that are not in the urban part of the city of Kingston.<br><br>
We may also evaluate which stops in the network are walkable. We find that 706 of 799 stops are within 200m 

In [55]:
stop_density = stop_distance < 200
walkable_stop = stop_density.sum(axis=0)>0
print(walkable_stop.sum())

706


The next section of code is trying to determine which routes service which stops and can be used to determine which stops serve as potential transfers between bus routes. A stop is said to be part of a bus route if the haversine formula measures a distance of \<200m of the shaded route to the stop. The routes are stored in a list for that record in the data frame.

In [35]:
# The calculation needs to be complete for all bus stops. Stops need to be recorded in the dataframe. TBD.
k = 0
for s in transit_stops['fields.stop_coordinates']:
    stop_test = np.array(s)
    route_list = list()
    for route, ind in enumerate(busroute['fields.shape.coordinates']):
        p = 0
        route_test = np.array(busroute['fields.shape.coordinates'][route][0])
        for ind in route_test:
            test_param = haversine(np.flip(route_test[p]),stop_test)
            if test_param<150:
                # Append bus route to list.
                route_list.append(busroute['fields.route_short_name'][route])
                # Loop breaks if stop identified as being within 100m of bus route.
                break


            p = p+1

    transit_stops['bus_routes'][k] = route_list
    # print(transit_stops['fields.stop_name'][k]," ",transit_stops['bus_routes'][k])
    k = k+1



c:\users\ryan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\ryan\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


The number of routes each stop services can be seen accross the network using a histogram. There are 38 stops which were not assigned to a route. This is likely due to the distcrete spacing of the GPS shape coordinates in the bus route feature. These can be binned with the stops that service a single route. What is pretty interesting is that there are 19 bus stops that have 10 bus routes within 200m of the stop.

In [44]:
print(np.histogram(transit_stops['bus_routes'].str.len()))

(array([ 38, 461, 113, 109,  22,  17,   7,   4,   9,  19], dtype=int64), array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]))


In [48]:
print(np.histogram(transit_stops['bus_routes'].str.len(),bins=[0,2,3,4,5,6,7,8,9,10]))

(array([499, 113, 109,  22,  17,   7,   4,   9,  19], dtype=int64), array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10]))


In [49]:
print(round(100*499/799),'% of the transit network does not have a transfer.')

62 % of the transit network does not have a transfer.


We can evaluate the October ridership transit data provided by the city of Kingston as an estimate for the daily usage of the transit network. We can see if there is a correlation between ridership (which stops riders are boarding) and distance from driveways to bus stops.

In [15]:
ridership = pd.read_excel('./Transit Data - October.xlsx',engine = 'openpyxl')

In [63]:
# Initialize a feature in ridership dataframe called stop_name
#ridership['stop_name'] = pd.Series()

if str(ridership['Bus'][0]) in transit_stops['bus_routes'][0]:
    print(True)
    
transit_stops.head()

,datasetid,recordid,record_timestamp,fields.stop_id,fields.stop_name,fields.stop_coordinates,fields.wheelchair_boarding,geometry.type,geometry.coordinates,bus_routes
0,transit-gtfs-stops,dc7bd0c8aafac3b95b52efe8b156deb1ac229524,2022-01-19T09:00:48.798000+00:00,00799,Norwest Road (east side of Midland),"[44.25722, -76.57395]",0,Point,"[-76.57395, 44.25722]","[6, 15, 7, 14, 501, 4, 10, COV]"
1,transit-gtfs-stops,8c6767e3195d529b6d3af49f57a34b5b8580fe0e,2022-01-19T09:00:48.798000+00:00,00850,Centennial Drive (south side of Cataraqui Woods),"[44.26926, -76.55933]",0,Point,"[-76.55933, 44.26926]",[14]
2,transit-gtfs-stops,15f9b959ea1f08f5318db7905927f9f740ecf9ab,2022-01-19T09:00:48.798000+00:00,09075,Crossfield Avenue (west side of Andersen),"[44.26475, -76.54892]",0,Point,"[-76.54892, 44.26475]",[14]
3,transit-gtfs-stops,5a71a7a20bc0985fe50a0837d38f553177f4545c,2022-01-19T09:00:48.798000+00:00,00858,Chadwick Parkette (west side of Andersen),"[44.26208, -76.54846]",0,Point,"[-76.54846, 44.26208]",[14]
4,transit-gtfs-stops,ec44e9770a1335c6fe80d0ef604dfca4ad792709,2022-01-19T09:00:48.798000+00:00,00407,Vanguard Court (east side of Waterloo),"[44.25257, -76.54781]",0,Point,"[-76.54781, 44.25257]",[14]


The following cell takes a long time to run. The loops iterate over the 700k rider boarding GPS coordinates with the 800 GPS coordinates in map stops dataframe. The following was done locally and saved to a copyWe can then evaluate which stops have the highest boarding rates. Bus route 17 is not in the busroute data from city of Kingston.

In [101]:
# The calculation needs to be complete for all bus stops. 
# Stops need to be recorded in the dataframe.
for ind in list(ridership.index):
    
    boarding_gps = [ridership['Latitude'][ind],ridership['Longitude'][ind]]
    
    # Continue call in cases where the gps coords is [0,0]
    if boarding_gps[0]==0:
        continue
        
    k = 0
    for s in transit_stops['fields.stop_coordinates']:
        if str(ridership['Route'][ind]) in transit_stops['bus_routes'][k]:
            stop_test = np.array(s)
            test_param = haversine(boarding_gps,stop_test)
            if test_param<20:
                # Append bus route to list.
                # Loop breaks if boarding coords identified as being within 20m of bus stop.
                ridership['stop_name'][ind] = transit_stops['fields.stop_id'][k]
                break
        k = k+1


c:\users\ryan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [102]:
ridership.to_excel('transit_update.xlsx')

In [108]:
ridership['stop_name'].value_counts()

S02037    22133
S02079    18092
S00501    14292
S00292    14076
S02036    10362
          ...  
09085         1
00598         1
00100         1
00834         1
00754         1
Name: stop_name, Length: 644, dtype: int64

In [142]:

time = pd.Series(ridership['Time'].dt.hour)
day = pd.Series(ridership['Date'].dt.day)
weekday = pd.Series(ridership['Date'].dt.dayofweek)
len(weekday[weekday>=5])/len(weekday)

0.16822205726013748

In [169]:
day_df = day.to_frame(name = 'day')
weekday_df = weekday.to_frame(name = 'weekday')
time_df = time.to_frame(name = 'time')
#summary_ridership = pd.Series([])
df = pd.concat([day_df,weekday_df,time_df],axis = 1)

In [173]:

pd.crosstab(index = df['weekday'],columns = df['time'])


time,0,1,2,3,5,6,7,8,9,10,...,14,15,16,17,18,19,20,21,22,23
weekday,,,,,,,,,,,,,,,,,,,,,
0,76,35,9,7,68,2363,7958,8881,6592,5998,...,10181,10651,10851,9971,6811,4657,3540,3067,1575,769
1,135,47,6,19,103,3042,10280,10743,8489,7338,...,12606,12707,12500,11537,8334,5163,4567,3485,1981,737
2,2,2,0,8,90,2507,8454,9126,6855,6654,...,9312,10022,9763,8817,6410,3898,3425,2872,1412,581
3,3,8,0,24,81,2490,8429,9237,6864,6134,...,9862,10160,10770,9662,6987,3964,3435,3074,1985,652
4,3,108,0,10,85,2236,7272,7782,6310,6054,...,9608,10079,10640,8748,6429,4441,3714,3426,2378,881
5,2,1,0,13,28,811,1369,2332,3000,3815,...,5792,6256,6134,5134,4582,3264,2744,2272,2219,892
6,261,231,32,2,14,354,585,1772,2391,3152,...,4999,5073,5024,4499,4021,2916,2134,2024,979,572


In [124]:
# take input Dates in a range
dates = pd.Series(pd.date_range('2019-8-5 10:23:05', periods = 6, freq ='H'))
  
# convert in a dict container
frame = pd.DataFrame(dict(givenDate = dates))
dates

0   2019-08-05 10:23:05
1   2019-08-05 11:23:05
2   2019-08-05 12:23:05
3   2019-08-05 13:23:05
4   2019-08-05 14:23:05
5   2019-08-05 15:23:05
dtype: datetime64[ns]